<a href="https://colab.research.google.com/github/funway/nid-imbalance-study/blob/main/preprocessing/20_02_2018csv_trimming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trimming Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
```
Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
  unique labels: [2], {'Benign': 7372557, 'DDoS attacks-LOIC-HTTP': 576191}
```

The dataset Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv is too large (more than 3GB) to be handled by Colab free tier in onetime processing. Also,
 and the performance of the free version of Colab cannot handle such a large amount of data at once. Also, there is too much **Benign** data, as shown above. Therefore, we can trim the CSV file to retain the attack data and some benign data.

In [ ]:
import os
from google.colab import drive
import pandas as pd

if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')

## 先手工将 csv 文件改名成 .bak 后缀 ##

csv_file = '/content/drive/MyDrive/NYIT/870/datasets/CSE-CIC-IDS2018/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv'
bak_file = csv_file + '.bak'
output_file = csv_file

file_size = os.path.getsize(bak_file)  # 获取文件大小，单位是字节
print(f'原始文件大小: {file_size / (1024 ** 2):.2f} MB')

chunksize = 1000000  # 每次读取 100,000 行
benign_count_per_chunk = []  # 记录每个 chunk 中 Benign 样本的数量
total_benign_count = 0  # 总的 Benign 样本数量

# 先分块读取一次，统计每个 chunk 中的 Benign 样本数量
for chunk in pd.read_csv(bak_file, chunksize=chunksize):
    # 统计 `Benign` 样本数量
    benign_data = chunk[chunk['Label'] == 'Benign']
    benign_count = benign_data.shape[0]
    benign_count_per_chunk.append(benign_count)
    total_benign_count += benign_count
    print(f'当前 chunk 中 Benign 样本数量: {benign_count}')
print(f'总的 Benign 样本数量: {total_benign_count}')

final_data = pd.DataFrame()  # 用于存储最终数据
target_benign_count = 500000  # 我们希望最终 `Benign` 样本的数量是 500,000
benign_sample_ratios = [count / total_benign_count for count in benign_count_per_chunk]  # 每个 chunk 中 `Benign` 样本的比例
print(f'每个 chunk 中 `Benign` 样本的比例: {benign_sample_ratios}')

# 第二次分块读取，从每个 chunk 中等比例抽取 Benign样本
# (但后面发现可能没必要，因为它原始文件中，Benign 样本也不是随机分布的，
# 除开第一个 chunk 有非 Benign 数据，剩下的 chunks 几乎100%全是 Benign 数据)
for chunk, ratio in zip(pd.read_csv(bak_file, chunksize=chunksize), benign_sample_ratios):
    benign_data = chunk[chunk['Label'] == 'Benign']
    non_benign_data = chunk[chunk['Label'] != 'Benign']

    # 计算该 chunk 中要抽取的 `Benign` 样本数量
    required_benign_count = int(target_benign_count * ratio)
    print(f'该 chunk 中要抽取的 `Benign` 样本数量: {required_benign_count}')

    # 从该 chunk 中抽取 `Benign` 样本
    sampled_benign_data = benign_data.sample(n=required_benign_count, random_state=42)
    final_data = pd.concat([final_data, sampled_benign_data, non_benign_data])
    pass

print(f"最终数据集中不同标签的样本数量: {final_data['Label'].value_counts()}")
final_data.to_csv(output_file, index=False)
print(f'最终数据已保存到: {output_file}')

Mounted at /content/drive
原始文件大小: 3867.08 MB
当前 chunk 中 Benign 样本数量: 423809
当前 chunk 中 Benign 样本数量: 1000000
当前 chunk 中 Benign 样本数量: 1000000
当前 chunk 中 Benign 样本数量: 1000000
当前 chunk 中 Benign 样本数量: 1000000
当前 chunk 中 Benign 样本数量: 1000000
当前 chunk 中 Benign 样本数量: 1000000
当前 chunk 中 Benign 样本数量: 948748
总的 Benign 样本数量: 7372557
每个 chunk 中 `Benign` 样本的比例: [0.057484669158882055, 0.1356381510512567, 0.1356381510512567, 0.1356381510512567, 0.1356381510512567, 0.1356381510512567, 0.1356381510512567, 0.1286864245335777]
该 chunk 中要抽取的 `Benign` 样本数量: 28742
该 chunk 中要抽取的 `Benign` 样本数量: 67819
该 chunk 中要抽取的 `Benign` 样本数量: 67819
该 chunk 中要抽取的 `Benign` 样本数量: 67819
该 chunk 中要抽取的 `Benign` 样本数量: 67819
该 chunk 中要抽取的 `Benign` 样本数量: 67819
该 chunk 中要抽取的 `Benign` 样本数量: 67819
该 chunk 中要抽取的 `Benign` 样本数量: 64343
最终数据集中不同标签的样本数量: Label
DDoS attacks-LOIC-HTTP    576191
Benign                    499999
Name: count, dtype: int64


In [ ]:
print('hello')

hello
